# Task 4 
Repeat TASK3, but now forthe scenario of the ray being refracted on a path between two media with differing wave speeds.
Demonstrate that Fermat’s principle of minimizing travel time implies Snell’s law of refraction.


## Extension Opportunities
Using the slider from the previous task, I could use this to demonstrate the "Brachistochrone curve" (the curve of fastest descent) be demonstrated by having blocks of increasing refractive indices. 
Somehow extract this curve (or prove that this is the correct curve (how? this seems very difficult to prove (but easy to demonstrate)))